In [216]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import pickle
import datetime as dt
from datetime import datetime
from datetime import timedelta
from pytz import timezone
import pytz
from pytz import all_timezones
import pickle

In [217]:
path="C:\\Users\\11147\\Desktop\\Sanket_backup\\Mhack\\"

In [218]:
df=pd.read_parquet(r"C:\Users\11147\Desktop\Sanket_backup\Mhack\drive\drive_1557398801351_trip_1557398302922_part-00000-tid-4787120893022696848-4897608f-5b93-42b5-a8ab-e90f5cadf0bf-1853-c000.snappy.parquet",engine="pyarrow")

In [219]:
drive_col=df.columns.tolist()
drive_col

['vehicle_id',
 'trip_id',
 'datetime',
 'velocity',
 'accel_x',
 'accel_y',
 'accel_z',
 'engine_coolant_temp',
 'eng_load',
 'fuel_level',
 'iat',
 'rpm']

In [220]:
df_drive=pd.DataFrame(columns=drive_col)

In [221]:
for j in os.listdir(path + "drive"):
    df=pd.read_parquet(path+"drive"+ "\\" +str(j),engine="pyarrow")
    df_drive=pd.concat([df_drive,df],ignore_index=True)

In [222]:
df_drive.shape

(9217528, 12)

In [223]:
df_drive.head()

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm
0,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:00,0.00,50.0,65.13,87.92,150.70,192.88,118.66,95.0,2107.44
1,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:01,47.49,51.0,72.34,83.78,154.76,196.04,116.69,100.0,2119.17
2,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:02,57.94,50.0,69.20,87.73,165.50,196.01,105.46,90.0,2112.32
3,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:03,57.50,48.0,70.83,84.95,157.32,193.31,110.35,90.0,2092.94
4,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 23:00:04,52.29,47.0,70.22,92.90,151.00,189.49,104.25,97.0,2117.87


In [224]:
df_drive.velocity=df_drive.velocity.apply(lambda x: x*5/18)

In [225]:
df_drive.datetime=df_drive.datetime.apply(lambda x :x-timedelta(hours=7)) 

In [226]:
target_df=pd.DataFrame(columns=["trip_id","ft_cnt_vehicle_deaccel_val","ft_cnt_vehicle_accel_val"])

In [227]:
total_trip_id=df_drive.trip_id.unique().tolist()

In [228]:
for trip in total_trip_id:
    tr=df_drive[df_drive.trip_id==trip]
    tr_sub=tr.velocity.tolist()
    variation=[]
    for i in range(0,len(tr_sub)-1):
        if tr_sub[i]>tr_sub[i+1]:
            variation.append("d")
        elif tr_sub[i]<tr_sub[i+1]:
            variation.append("a")
    if len(variation)==0:
        deacc_val=0
        acc_val=0
    else:
        change=[]
        for i in range(0,len(variation)-1):
            if variation[i]==variation[i+1]:
                pass
            else:
                change.append(variation[i])
        if variation[-1]==variation[-2]:
            pass
        else:
            change.append(variation[-1])
        deacc_val=change.count("d")
        acc_val=change.count("a")
    target_df=target_df.append({"trip_id":trip,"ft_cnt_vehicle_deaccel_val":change.count("d"),"ft_cnt_vehicle_accel_val":change.count("a")},ignore_index=True)

In [25]:
pickle_in = open("data.pickle","rb")
target_df = pickle.load(pickle_in)

In [229]:
target_df.head()

,trip_id,ft_cnt_vehicle_deaccel_val,ft_cnt_vehicle_accel_val
0,db3c08b01205459294bbc91e3f7dfeb6,1429,1430
1,77b52511437f4b78871eb1ef6080daae,2381,2382
2,b086a49b14b4420c98bbb569510c5629,894,895
3,726358cae8754f52a9fcdd0d1441d618,213,214
4,1db28627c3c748868f4eb7c6afb8d083,1334,1335


In [230]:
#Hard accel or deaacel

In [232]:
deacc=[]

In [233]:
for trip in total_trip_id:
    speed_var=[]
    tr=df_drive[df_drive.trip_id==trip]
    tr_sub=tr.velocity.tolist()
    for speed in range(0,len(tr_sub)-1):
        speed_var.append(tr_sub[speed+1]-tr_sub[speed])
    if len(speed_var)==1:
        dec_10=0
    else:
        new_speed_deaccel=[x if x<=-10 else "no" for x in speed_var]
        hard_deaccel=[]
        for i in range(0,len(new_speed_deaccel)-1):
            if new_speed_deaccel[i]==new_speed_deaccel[i+1]:
                pass
            else:
                hard_deaccel.append(new_speed_deaccel[i])
        if new_speed_deaccel[-1]==new_speed_deaccel[-2]:
            pass
        else:
            hard_deaccel.append(new_speed_deaccel[-1])

        hard_deaccel.append("no")
        hard_deaccel.insert(0,"no")
        
        m=[]
        z=0
        for i in range(0,len(hard_deaccel)-1):
            if type(hard_deaccel[i])!=type(hard_deaccel[i+1]):
                if type(hard_deaccel[i])==int and type(hard_deaccel[i-1])==int:
                    pass
                else:
                    m.append(hard_deaccel[i])
                z=0
            elif type(hard_deaccel[i])==type(hard_deaccel[i+1])==int:
                v=[z,hard_deaccel[i],hard_deaccel[i+1]]
                z=max(v)
                if hard_deaccel[i+2]=="n":
                    m.append(z)
            else:
                pass
            
            dec_10=sum(isinstance(x, float) for x in m)
    deacc.append(dec_10)

In [234]:
target_df["ft_sum_hard_brakes_10_flg_val"]=deacc

In [235]:
deacc_3_10=[]

In [ ]:
for trip in total_trip_id:
    speed_var=[]
    tr=df_drive[df_drive.trip_id==trip]
    tr_sub=tr.velocity.tolist()
    for speed in range(0,len(tr_sub)-1):
        speed_var.append(tr_sub[speed+1]-tr_sub[speed])
    if len(speed_var)==1:
        dec_10=0
    else:
        new_speed_deaccel=[x if -3>= x >-10 else "no" for x in speed_var]
        hard_deaccel=[]
        for i in range(0,len(new_speed_deaccel)-1):
            if new_speed_deaccel[i]==new_speed_deaccel[i+1]:
                pass
            else:
                hard_deaccel.append(new_speed_deaccel[i])
        if new_speed_deaccel[-1]==new_speed_deaccel[-2]:
            pass
        else:
            hard_deaccel.append(new_speed_deaccel[-1])

        hard_deaccel.append("no")
        hard_deaccel.insert(0,"no")
        
        m=[]
        z=0
        for i in range(0,len(hard_deaccel)-1):
            if type(hard_deaccel[i])!=type(hard_deaccel[i+1]):
                if type(hard_deaccel[i])==int and type(hard_deaccel[i-1])==int:
                    pass
                else:
                    m.append(hard_deaccel[i])
                z=0
            elif type(hard_deaccel[i])==type(hard_deaccel[i+1])==int:
                v=[z,hard_deaccel[i],hard_deaccel[i+1]]
                z=max(v)
                if hard_deaccel[i+2]=="n":
                    m.append(z)
            else:
                pass
            
            dec_10=sum(isinstance(x, float) for x in m)
    deacc_3_10.append(dec_10)

In [ ]:
target_df["ft_sum_hard_brakes_3_flg_val"]=deacc_3_10

In [ ]:
acc=[]

In [ ]:
for trip in total_trip_id:
    speed_var=[]
    tr=df_drive[df_drive.trip_id==trip]
    tr_sub=tr.velocity.tolist()
    for speed in range(0,len(tr_sub)-1):
        speed_var.append(tr_sub[speed+1]-tr_sub[speed])
    if len(speed_var)==1:
        acc_10=0
    else:
        new_speed_accel=[x if x > 10 else "no" for x in speed_var]
        hard_accel=[]
        for i in range(0,len(new_speed_accel)-1):
            if new_speed_accel[i]==new_speed_accel[i+1]:
                pass
            else:
                hard_accel.append(new_speed_accel[i])
        if new_speed_accel[-1]==new_speed_accel[-2]:
            pass
        else:
            hard_accel.append(new_speed_accel[-1])

        hard_accel.append("no")
        hard_accel.insert(0,"no")
        
        m=[]
        z=0
        for i in range(0,len(hard_accel)-1):
            if type(hard_accel[i])!=type(hard_accel[i+1]):
                if type(hard_accel[i])==int and type(hard_accel[i-1])==int:
                    pass
                else:
                    m.append(hard_accel[i])
                z=0
            elif type(hard_accel[i])==type(hard_accel[i+1])==int:
                v=[z,hard_accel[i],hard_accel[i+1]]
                z=max(v)
                if hard_accel[i+2]=="n":
                    m.append(z)
            else:
                pass
            
            acc_10=sum(isinstance(x, float) for x in m)
    acc.append(acc_10)

In [ ]:
target_df["ft_sum_hard_accel_10_flg_val"]=acc

In [ ]:
acc_3_10=[]

In [ ]:
for trip in total_trip_id:
    speed_var=[]
    tr=df_drive[df_drive.trip_id==trip]
    tr_sub=tr.velocity.tolist()
    for speed in range(0,len(tr_sub)-1):
        speed_var.append(tr_sub[speed+1]-tr_sub[speed])
    if len(speed_var)==1:
        acc_10=0
    else:
        new_speed_accel=[x if 3<= x < 10 else "no" for x in speed_var]
        hard_accel=[]
        for i in range(0,len(new_speed_accel)-1):
            if new_speed_accel[i]==new_speed_accel[i+1]:
                pass
            else:
                hard_accel.append(new_speed_accel[i])
        if new_speed_accel[-1]==new_speed_accel[-2]:
            pass
        else:
            hard_accel.append(new_speed_accel[-1])

        hard_accel.append("no")
        hard_accel.insert(0,"no")
        
        m=[]
        z=0
        for i in range(0,len(hard_accel)-1):
            if type(hard_accel[i])!=type(hard_accel[i+1]):
                if type(hard_accel[i])==int and type(hard_accel[i-1])==int:
                    pass
                else:
                    m.append(hard_accel[i])
                z=0
            elif type(hard_accel[i])==type(hard_accel[i+1])==int:
                v=[z,hard_accel[i],hard_accel[i+1]]
                z=max(v)
                if hard_accel[i+2]=="n":
                    m.append(z)
            else:
                pass
            
            acc_10=sum(isinstance(x, float) for x in m)
    acc_3_10.append(acc_10)

In [ ]:
target_df["ft_sum_hard_accel_3_flg_val"]=acc_3_10

In [ ]:
target_df.head(10)

In [ ]:
trip_acl=[]
trip_dcl=[]

In [ ]:
for trip in total_trip_id:
    acl=0
    decl=0
    tr=df_drive[df_drive.trip_id==trip]
    tr_sub=tr.velocity.tolist()
    if sum(tr_sub)==0:
        acl+=0
        decl+=0
    else:
        for i in range(0,len(tr_sub)-1):
            if tr_sub[i]<tr_sub[i+1]:
                acl+=1
            else:
                decl+=1
    trip_acl.append(acl)
    trip_dcl.append(decl)
    

In [ ]:
target_df["ft_sum_time_deaccel_val"]=trip_dcl
target_df["ft_sum_time_accel_val"]=trip_acl

In [ ]:
len(trip_acl),len(trip_dcl)

In [ ]:
target_df.head(10)

In [ ]:
pickle_out = open("data.pickle","wb")
pickle.dump(target_df, pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("data.pickle","rb")
target_df = pickle.load(pickle_in)

In [ ]:
target_df.head()

In [81]:
target_df.to_csv("drive_features.csv",index=False)

In [9]:
################################################################################################################################

Engine csv

In [84]:
df_drive.head()

,vehicle_id,trip_id,datetime,velocity,accel_x,accel_y,accel_z,engine_coolant_temp,eng_load,fuel_level,iat,rpm
0,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 16:00:00,0.000000,50.0,65.13,87.92,150.70,192.88,118.66,95.0,2107.44
1,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 16:00:01,13.191667,51.0,72.34,83.78,154.76,196.04,116.69,100.0,2119.17
2,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 16:00:02,16.094444,50.0,69.20,87.73,165.50,196.01,105.46,90.0,2112.32
3,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 16:00:03,15.972222,48.0,70.83,84.95,157.32,193.31,110.35,90.0,2092.94
4,1000515,db3c08b01205459294bbc91e3f7dfeb6,2017-01-06 16:00:04,14.525000,47.0,70.22,92.90,151.00,189.49,104.25,97.0,2117.87


In [94]:
df_vehicle=pd.read_csv(r"C:\Users\11147\Desktop\Sanket_backup\Mhack\vehicle.csv")

In [95]:
df_vehicle

,vehicle_id,year,make,Model,drivetrain,max_torque,max_horsepower,max_horsepower_rpm,max_torque_rpm,engine_displacement,fuel_type,fuel_tank_capacity,fuel_economy_city,fuel_economy_highway,cylinders,forced_induction,device_generation
0,1000500,2016,Honda,Civic,2,174,140,6500,1500,1.799,1059,47,16.50,20.00,4,1054.0,5
1,1000501,2016,Jeep,Compass,4,350,171,3750,1250,1.956,1059,60,17.10,21.20,3,NaN,4
2,1000502,2016,Hyundai,Creta,2,260,126,4000,1500,1.582,1059,55,19.67,24.10,4,1054.0,3
3,1000503,2016,Skoda,Superb,2,250,177,5100,1750,1.798,1059,66,14.67,23.30,3,NaN,2
4,1000504,2017,Volkswagen,Passat,4,350,174,3600,1500,1.968,1059,66,17.42,20.43,4,NaN,4
5,1000506,2017,BMW,3 Series GT,2,400,188,4000,1800,1.995,1059,57,21.76,26.32,3,NaN,3
6,1000507,2017,Audi,Q3,3,250,148,5000,1400,1.395,1059,64,16.90,21.40,3,1054.0,2


In [85]:
vehicle=df_drive.vehicle_id.unique().tolist()

In [86]:
engine=df_drive[["vehicle_id","eng_load","rpm","datetime"]]

In [87]:
engine.sort_values("vehicle_id",inplace=True)    

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [88]:
engine["torque_utilization"]=engine["eng_load"]/255

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [89]:
engine["max_torque"]=0
engine["max_hp"]=0
engine["max_hp_rpm"]=0

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [130]:
for veh in vehicle:
    if veh in (df_vehicle.vehicle_id.tolist()):
        m_t=df_vehicle[df_vehicle.vehicle_id==veh]["max_torque"].values
        m_h=df_vehicle[df_vehicle.vehicle_id==veh]["max_horsepower"].values
        m_h_r=df_vehicle[df_vehicle.vehicle_id==veh]["max_horsepower_rpm"].values
        engine.loc[engine.vehicle_id==veh,"max_torque"]=m_t[0]
        engine.loc[engine.vehicle_id==veh,"max_hp"]=m_h[0]
        engine.loc[engine.vehicle_id==veh,"max_hp_rpm"]=m_h_r[0]
    else:
        pass

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
engine.drop(["hp_uti","tor_uti","rpm_uti"],axis=1,inplace=True)

In [ ]:
engine.drop("active_hp",axis=1,inplace=True)

In [ ]:
engine.drop("torque_utilization",axis=1,inplace=True)

In [131]:
engine_with=engine.loc[engine["vehicle_id"].isin(df_vehicle.vehicle_id.tolist())]

In [132]:
engine_without=engine.loc[~engine["vehicle_id"].isin(df_vehicle.vehicle_id.tolist())]

In [133]:
engine_without.head()

,vehicle_id,eng_load,rpm,datetime,torque_utilization,max_torque,max_hp,max_hp_rpm
8111126,1000505,218.35,2141.09,2017-01-22 10:46:12,0.856275,0,0,0
8111122,1000505,212.48,2143.20,2017-01-22 10:46:08,0.833255,0,0,0
8111125,1000505,204.92,2141.11,2017-01-22 10:46:11,0.803608,0,0,0
8111124,1000505,210.39,2142.20,2017-01-22 10:46:10,0.825059,0,0,0
8111123,1000505,215.55,2148.21,2017-01-22 10:46:09,0.845294,0,0,0


In [136]:
engine_with.head()

,vehicle_id,eng_load,rpm,datetime,torque_utilization,max_torque,max_hp,max_hp_rpm,active_hp
3785942,1000500,194.21,2105.21,2017-01-06 14:09:53,0.761608,174,140,6500,53.119180
396088,1000500,199.97,1976.16,2017-01-06 15:22:33,0.784196,174,140,6500,51.341825
396087,1000500,205.29,1974.85,2017-01-06 15:22:32,0.805059,174,140,6500,52.672782
396086,1000500,193.45,1976.65,2017-01-06 15:22:31,0.758627,174,140,6500,49.680146
396085,1000500,210.08,1976.78,2017-01-06 15:22:30,0.823843,174,140,6500,53.954466


In [135]:
engine_with["active_hp"]=engine_with["eng_load"]/255 * engine_with["max_torque"] * engine_with["rpm"] / 5252

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [137]:
engine_with["hp_utl"]=engine_with["active_hp"] / engine_with["max_hp"]

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [138]:
engine_with["tor_utl"]=engine_with["eng_load"]/255

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [139]:
engine_with["rpm_uti"]=engine_with["rpm"]/engine_with["max_hp_rpm"]

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [140]:
engine_without["active_hp"]=0

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [141]:
engine_without["hp_utl"]=0

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [142]:
engine_without["tor_utl"]=engine_without["eng_load"]/255

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [143]:
engine_without["rpm_uti"]=0

C:\Users\11147\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [144]:
engine_with.shape

(3289653, 12)

In [145]:
engine_without.shape

(5927875, 12)

In [146]:
new_engine=pd.concat([engine_with,engine_without],ignore_index=True)

In [147]:
pickle_out = open("engine.pickle","wb")
pickle.dump(new_engine, pickle_out)
pickle_out.close()

In [148]:
pickle_in = open("engine.pickle","rb")
new_engine = pickle.load(pickle_in)

In [149]:
new_engine=new_engine.sort_values(["vehicle_id","datetime"])

In [114]:
target_df_engine=pd.DataFrame(columns=["a"])
ids=[]
t_60=[]
t_70=[]
t_80=[]
t_90=[]
h_50=[]
h_60=[]
h_70=[]
h_80=[]
rpm_50=[]
rpm_60=[]


for i in new_engine.vehicle_id.unique().tolist():
    ids.append(i)
    a=new_engine[new_engine.vehicle_id==i]
    t_60.append(len(a[(a.tor_utl>=0.60) & (a.tor_utl<0.7)]))
    t_70.append(len(a[(a.tor_utl>=0.70) & (a.tor_utl<0.8)]))
    t_80.append(len(a[(a.tor_utl>=0.80) & (a.tor_utl<0.9)]))
    t_90.append(len(a[(a.tor_utl>=0.90) & (a.tor_utl<1)]))
    h_50.append(len(a[(a.hp_utl>=0.50) & (a.hp_utl<0.6)]))
    h_60.append(len(a[(a.hp_utl>=0.60) & (a.hp_utl<0.7)]))
    h_70.append(len(a[(a.hp_utl>=0.70) & (a.hp_utl<0.8)]))
    h_80.append(len(a[(a.hp_utl>=0.80) & (a.hp_utl<0.9)]))
    rpm_50.append(len(a[(a.rpm_uti>=0.50) & (a.rpm_uti<0.60)]))
    rpm_60.append(len(a[(a.rpm_uti>=0.60) & (a.rpm_uti<0.70)]))

In [115]:
target_df_engine["vehicle_id"]=ids
target_df_engine["ft_torque_util_60pct_s"]=t_60
target_df_engine["ft_torque_util_70pct_s"]=t_70
target_df_engine["ft_torque_util_80pct_s"]=t_80
target_df_engine["ft_torque_util_90pct_s"]=t_90
target_df_engine["ft_horsepower_util_50pct_s"]=h_50
target_df_engine["ft_horsepower_util_60pct_s"]=h_60
target_df_engine["ft_horsepower_util_70pct_s"]=h_70
target_df_engine["ft_horsepower_util_80pct_s"]=h_80
target_df_engine["ft_rpm_util_50pct_s"]=rpm_50
target_df_engine["ft_rpm_util_60pct_s"]=rpm_60

target_df_engine.drop("a",axis=1,inplace=True)

In [185]:
a=new_engine.groupby(["vehicle_id",pd.Grouper(key='datetime', freq='W-MON')]).apply(lambda df : df[(df.tor_utl>0.6) & (df.tor_utl<0.7)]["rpm_uti"].count()).reset_index()
b=new_engine.groupby(["vehicle_id",pd.Grouper(key='datetime', freq='W-MON')]).apply(lambda df : df[(df.tor_utl>0.7) & (df.tor_utl<0.8)]["rpm_uti"].count()).reset_index()
c=new_engine.groupby(["vehicle_id",pd.Grouper(key='datetime', freq='W-MON')]).apply(lambda df : df[(df.tor_utl>0.8) & (df.tor_utl<0.9)]["rpm_uti"].count()).reset_index()
d=new_engine.groupby(["vehicle_id",pd.Grouper(key='datetime', freq='W-MON')]).apply(lambda df : df[(df.tor_utl>0.9) & (df.tor_utl<1.0)]["rpm_uti"].count()).reset_index()

In [199]:
a.rename(columns={0:"ft_torque_util_60pct_s"},inplace=True)
a["ft_torque_util_70pct_s"]=b[0]
a["ft_torque_util_80pct_s"]=c[0]
a["ft_torque_util_90pct_s"]=d[0]

In [202]:
e=new_engine.groupby(["vehicle_id",pd.Grouper(key='datetime', freq='W-MON')]).apply(lambda df : df[(df.hp_utl>0.5) & (df.hp_utl<0.6)]["rpm_uti"].count()).reset_index()
f=new_engine.groupby(["vehicle_id",pd.Grouper(key='datetime', freq='W-MON')]).apply(lambda df : df[(df.hp_utl>0.6) & (df.hp_utl<0.7)]["rpm_uti"].count()).reset_index()
g=new_engine.groupby(["vehicle_id",pd.Grouper(key='datetime', freq='W-MON')]).apply(lambda df : df[(df.hp_utl>0.7) & (df.hp_utl<0.8)]["rpm_uti"].count()).reset_index()
h=new_engine.groupby(["vehicle_id",pd.Grouper(key='datetime', freq='W-MON')]).apply(lambda df : df[(df.hp_utl>0.8) & (df.hp_utl<0.9)]["rpm_uti"].count()).reset_index()

In [205]:
a["ft_horsepower_util_50pct_s"]=e[0]
a["ft_horsepower_util_60pct_s"]=f[0]
a["ft_horsepower_util_70pct_s"]=g[0]
a["ft_horsepower_util_80pct_s"]=h[0]

In [209]:
i=new_engine.groupby(["vehicle_id",pd.Grouper(key='datetime', freq='W-MON')]).apply(lambda df : df[(df.rpm_uti>0.5) & (df.rpm_uti<0.6)]["active_hp"].count()).reset_index()
j=new_engine.groupby(["vehicle_id",pd.Grouper(key='datetime', freq='W-MON')]).apply(lambda df : df[(df.rpm_uti>0.6) & (df.rpm_uti<0.7)]["active_hp"].count()).reset_index()

In [210]:
a["ft_rpm_util_50pct_s"]=i[0]
a["ft_rpm_util_60pct_s"]=j[0]

In [212]:
a.rename(columns={"datetime":"week_start_date"},inplace=True)

In [214]:
a.to_csv("engine_features.csv",index=False)